In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/diamonds/diamonds.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['cut'].unique()

In [ ]:
df['color'].unique()

In [ ]:
df['clarity'].unique()

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df['price'] = df['price'].astype(float)

In [ ]:
df.info()

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.scatterplot(data=df, x='carat', y='price');

In [ ]:
sns.scatterplot(data=df, x='depth', y='price');

In [ ]:
sns.scatterplot(data=df, x='table', y='price');

In [ ]:
sns.displot(data=df, x='price')

In [ ]:
sns.boxplot(data=df, x='price')

In [ ]:
df.describe()

Removing outliers 1.5 * iqr = 10493

In [ ]:
df = df[df['price'] < 10493]

In [ ]:
X = df.drop('price', axis=1)

In [ ]:
y = df['price']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
train_rmse_erros = []
test_rmse_erros = []

for d in range(1,4):
    
    poly_converter = PolynomialFeatures(degree=d, include_bias=False)
    poly_features = poly_converter.fit_transform(X)
   
    X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)
    
    model = LinearRegression()
    model.fit(X_train,y_train)
    
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    
    train_rmse_erros.append(train_rmse)
    test_rmse_erros.append(test_rmse)

In [ ]:
train_rmse_erros

In [ ]:
test_rmse_erros

In [ ]:
plt.plot(range(1,4),train_rmse_erros,label='Train RMES' )
plt.plot(range(1,4),test_rmse_erros,label='Test RMES' )
plt.xlabel('Degree of Poly')
plt.ylabel('RMSE')
plt.legend();

The best polynomial degree to avoid overfitting is 2

In [ ]:
polynomial_converter = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
polynomial_converter.fit(X)

In [ ]:
poly_features = polynomial_converter.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
MSE = mean_squared_error(y_test, y_pred)

In [ ]:
RMSE = np.sqrt(MSE)

In [ ]:
RMSE

In [ ]:
residual = y_test - y_pred

In [ ]:
sns.scatterplot(x=y_test, y=residual)
plt.axhline(y=0, color='red', ls='--')

In [ ]:
sns.displot(residual, bins=100, kde=True)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test, y_pred)

The diamond prices are affected not just by the features related in this data set, but there is another ones that have high influence over the final price, what are responsible to create the price outliers. Removing these outliers become the model able to predict the final price.